# **Multilingual Sports Commentary**

This notebook demonstrates how to build an end-to-end pipeline for generating, translating, and vocalizing sports commentary using Sarvam AI's suite of models.

### **Use Case**
Automating regional language broadcasts for global sports events. 
1. **Generate:** Use **Sarvam-M** (24B) to create high-energy commentary text.
2. **Translate:** Use **Mayura:v1** to convert that text into regional Indian languages.
3. **Vocalize:** Use **Bulbul:v3** to generate broadcast-ready audio output.

### **1. Prerequisites / Installation**

In [ ]:
!pip install -Uqq sarvamai

### **2. Import Libraries**

In [ ]:
import base64
from sarvamai import SarvamAI
from IPython.display import Audio, display

### **3. API Key Setup**

To use this notebook, obtain an API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai).

In [ ]:
SARVAM_API_KEY = "YOUR_SARVAM_API_KEY"

### **4. Client Initialization**

In [ ]:
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

### **5. Parameter Reference**

| Parameter | Type | Required | Options/Range | Description |
| :--- | :--- | :--- | :--- | :--- |
| `input` | string | Yes | Max 1000 chars | The text to be processed. |
| `source_language_code` | string | Yes | en-IN, hi-IN, etc. | BCP-47 language code. |
| `target_language_code` | string | Yes | te-IN, hi-IN, etc. | The output regional language. |
| `speaker` | string | Yes | shubh, priya, etc. | The voice personality for TTS. |
| `model` | string | Yes | sarvam-m, bulbul:v3 | The specific model to invoke. |

### **6. The AI-Powered Sports Pipeline**

#### **Step 6.1: Commentary Generation**
We use Sarvam-M to act as a commentator for an exciting match moment.

In [ ]:
system_prompt = "You are an enthusiastic football commentator. Generate a short, 1-sentence, high-energy commentary for a spectacular goal scored by Arsenal."

chat_response = client.chat.completions(
    model="sarvam-m",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "The score is 0-0 in the 90th minute, and Martin Ødegaard just hit a screamer from outside the box!"}
    ]
)

generated_commentary = chat_response.choices[0].message.content
print(f"Generated English Commentary: {generated_commentary}")

#### **Step 6.2: Multilingual Loop (Translate & Speak)**
We now loop through target languages to translate the AI's output and generate playable audio files.

In [ ]:
target_languages = ["te-IN", "hi-IN", "ta-IN"] # Telugu, Hindi, Tamil

for lang in target_languages:
    # 1. Translate
    trans_res = client.text.translate(
        input=generated_commentary,
        source_language_code="en-IN",
        target_language_code=lang,
        model="mayura:v1"
    )
    translated_text = trans_res.translated_text
    
    # 2. Text-to-Speech
    tts_res = client.text_to_speech.convert(
        input=translated_text,
        target_language_code=lang,
        model="bulbul:v3",
        speaker="shubh",
        speech_sample_rate=24000
    )
    
    # 3. Handle Audio Data
    audio_bytes = base64.b64decode(tts_res.audios[0])
    filename = f"commentary_{lang}.wav"
    with open(filename, "wb") as f:
        f.write(audio_bytes)
        
    print(f"\n--- {lang} Broadcast ---")
    print(f"Text: {translated_text}")
    display(Audio(filename, rate=24000))

### **7. Error Handling**

| Error Code | HTTP Status | Cause | Solution |
| :--- | :--- | :--- | :--- |
| `invalid_api_key_error` | 403 | Invalid API key | Verify your key in the Sarvam Dashboard. |
| `insufficient_quota_error` | 429 | Quota exceeded | Check your usage limits. |
| `internal_server_error` | 500 | Server-side issue | Wait and retry your request. |

### **8. Conclusion & Resources**

This recipe demonstrates how to chain multiple AI capabilities into a single, cohesive media-tech application. 

* [Sarvam AI Docs](https://docs.sarvam.ai)
* [Indic Language Support List](https://docs.sarvam.ai/language-support)

**Keep Building! 🚀**